In [2]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
#import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



In [3]:

########  dictionary aggregated info by folder                                 
#pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_new_COMPLETE.pickle'


pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_.pickle'
with open(pickle_name, 'rb') as handle:
    dict_folder_id_folder_attr = pickle.load(handle) 
print "num folders",len(dict_folder_id_folder_attr.keys())




########  dictionary aggregated info by user
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_new_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_user_attr = pickle.load(handle)
print "num users",len(dict_user_id_user_attr.keys())






########  dictionary folder list members
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_list_users_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_folder_id_list_users = pickle.load(handle) 
print "num users",len(dict_folder_id_list_users.keys())


########  dictionary user list folders
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_list_folders_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_list_folders = pickle.load(handle) 
print "num folders",len(dict_folder_id_list_users.keys())






#########  csv  all act aggr by date
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'DROPBOX_only_activity_aggr_by_date_COMPLETE.csv'
df_all_act = pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"], parse_dates=['folder_creation_date','date'])#, nrows=1000000) # set header=0 if i wanna pass it my own list of header names
df_all_act.drop('Unnamed: 0', axis=1, inplace=True)
print "num. users:",len(df_all_act.user_id.unique()),df_all_act.shape    # 200000 (3072489, 8)
  
df_all_act=df_all_act.replace('nan', np.nan)  ## for some reason, there are diff nomenclatures for NANs, and it is not interpreting them correctly when reading the file 
df_all_act=df_all_act.replace('NAN', np.nan)
df_all_act=df_all_act.replace('NaN', np.nan)
df_all_act=df_all_act.replace('-1', np.nan)
df_all_act=df_all_act.replace('', np.nan)


#convert the date columns from str to datetime    (for some reason, parsing when i read it does nothing)
df_all_act['date'] = pd.to_datetime(df_all_act['date'], errors='coerce')
df_all_act['folder_creation_date'] = pd.to_datetime(df_all_act['folder_creation_date'], errors='coerce')

df_all_act_no_NANs= df_all_act[df_all_act.date.notnull()]    
print "after removing rows without dates:\n  num. users:",len(df_all_act_no_NANs.user_id.unique()),df_all_act_no_NANs.shape    # 400000  (5141758, 7)


num folders 519045
num users 438284
num users 519045
num folders 519045
num. users: 438284 (6527213, 7)
after removing rows without dates:
  num. users: 175587 (5599544, 7)


In [ ]:
df_all_act.shape
print "after removing rows without dates:\n  num. users:",len(df_all_act_no_NANs.user_id.unique()),df_all_act_no_NANs.shape    # 400000  (5141758, 7)


In [4]:

def remove_nans(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
            pass            
            
            
    #print aux_lista   
    return aux_lista


############################



def remove_nans_for_strings(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
             aux_lista.append(lista[i])#pass             
            
            
    #print aux_lista   
    return aux_lista

 #pd.NaT(lista[i],dtype='datetime64[D]' ) == True:                                                             
               # numpy.datetime64('NaT')
                
#                np.isnat(np.array(['nat', 1, 2, 3, 4, 'nat', 5], dtype='datetime64[D]'))


############################



def remove_nans_for_dates(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            #if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
            if np.NaT(lista[i]) == True: 
               
                
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
             aux_lista.append(lista[i])#pass             
            
            
    #print aux_lista   
    return aux_lista




###############

def remove_nans_replace_by_zeros(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                aux_lista.append(0.)        #pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(float(lista[i]))
            #print "no problem"
        except : ## whenever it is a STR            
            pass             
            
            
    #print aux_lista   
    return aux_lista




###### get dict with  folder_id lifespan:
def remove_nans_replace_intervals_with_starting_value(lista):
    #print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when/ nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
        except TypeError: # (when not-nan):
            #print lista[i]   
            try:  
                aux_lista.append(float(lista[i]))
                #print "no problem"
            except ValueError:  # sometimes, the ranking is 75-82
                #print "issue found"
                value=lista[i]
#                 v1=float(value.split("-")[0])
#                 v2=float(value.split("-")[1])
                
                new_value=float(value.split("-")[0])#np.median([v1,v2])  
                aux_lista.append(new_value)
            
    #print aux_lista   
    return aux_lista


#############



def gini(old_list_of_values):
    
    list_of_values=[]
    for item in old_list_of_values:
        try:
            int(item) # if it is a NAN, it with fail
            list_of_values.append(item)
        except:
            pass
    
   # print list_of_values
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    try:
        return (fair_area - area) / fair_area
    except ZeroDivisionError:
#         print "problems with:",list_of_values   # if lista=[0,0,0]  or [0]
#         raw_input()
        return np.nan


##############


def  effective_num(lista_values):  # i can use it for the effective number of members in a folder, by activity, or the effective number of folders for a user, also by activity
    
     # first, i need to remove the zeros, because i cant do the log(0), but i can remove them, they dont count as effective members
    
    
    cont_num_nonzero_items=0
    H=0.
    tot_sum=float(sum(lista_values))
    for item in lista_values:
        if item >0:
            aux= item/tot_sum * np.log2( item/tot_sum)
            H += aux
            cont_num_nonzero_items +=1
            
    H = -1.0 * H
    
    eff_number = np.power(2.0, H)
    
    if cont_num_nonzero_items ==0:   # if the list of act is [0] then i want the eff. number to be 0, not 1
        eff_number =0
    
    return eff_number
        

In [5]:
########### i get dict for the folder_id time-related attr.

#df_all_act_no_NANs

def  temporary_grouping_folder(folder_id, dict_folder_id_folder_attr, df_select_folder_act, list_users) :  # df_select_folder_act is sorted by user_id and then date                 
        
    
    
    list_days_present_members=[]
    for user_id in list_users:
        
        df_act_user= df_select_folder_act[df_select_folder_act['user_id'] == user_id]                                   
        num_days= len(df_act_user.date.unique())                       
        list_days_present_members.append(num_days)
                            
    
    dict_folder_id_folder_attr[folder_id]["eff_num_members_by_days_present"]=effective_num(remove_nans_replace_by_zeros(list_days_present_members))
    
    
           
        


    
    
  
    
    
############################################
#############################################
     
cont=1

for folder_id in tqdm_notebook(dict_folder_id_list_users):  
  
    list_users=dict_folder_id_list_users[folder_id]

    
    
    df_select_folder_act=df_all_act_no_NANs[df_all_act_no_NANs['folder_id']== folder_id].sort_values(by='date')   
    
    #df_select_folder_act=df_all_act[df_all_act['folder_id']== folder_id].sort_values(by='date')        

    
    
   
   

    cont +=1
   # if len(df_select_folder_act)>0:
    temporary_grouping_folder(folder_id, dict_folder_id_folder_attr, df_select_folder_act, list_users)


#     if cont >= 10000:
#         break          
  
    
print "done"





#dict_folder_id_folder_attr['folder_id']["gini_num_days_present_over_members"]
    



pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_folder_id_folder_attr, handle)
print "written:", pickle_name




df_from_dict = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')
## add foler_id from index
df_from_dict.to_csv(pickle_name.replace("dict_",'').strip(".pickle")+".csv", sep=';')
print "written:", pickle_name.replace("dict_","").strip(".pickle")+".csv"



done
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr_.pickle
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/folder_id_folder_attr_with_time_attr_.csv


In [ ]:
########### i get dict for the folder_id time-related attr.

#df_all_act_no_NANs

def  grouping_folder(folder_id, dict_folder_id_folder_attr, df_select_folder_act, list_users) :  # df_select_folder_act is sorted by user_id and then date                 
    
#     print 
#     print df_select_folder_act
    
    dict_folder_id_folder_attr[folder_id]["act_per_day"] = float(dict_folder_id_folder_attr[folder_id]["folder_tot_act"]) / dict_folder_id_folder_attr[folder_id]["act_period"]
    
    dict_folder_id_folder_attr[folder_id]["act_per_day_per_active_user"] = float(dict_folder_id_folder_attr[folder_id]["folder_tot_act"]) / \
    (dict_folder_id_folder_attr[folder_id]["act_period"]*dict_folder_id_folder_attr[folder_id]["number_active_members"])
    
    
    dict_folder_id_folder_attr[folder_id]["overall_num_active_days"]=len(df_select_folder_act.date.dropna().unique())   #  there shouldnt be any nans anyway len(remove_nans_for_dates(df_select_folder_act.date.unique()))
    
                                        
           
    tot_num_days=dict_folder_id_folder_attr[folder_id]["act_period"]
    
    #rolling_window=int(tot_num_days/10.)
    
    
    dict_folder_id_folder_attr[folder_id]["num_time_stamps"]=len(df_select_folder_act)
    
    
    
   
    list_days_present_members=[]
    for user_id in list_users:
        
        df_act_user= df_select_folder_act[df_select_folder_act['user_id'] == user_id]                                   
        num_days= len(df_act_user.date.unique())                       
        list_days_present_members.append(num_days)
        
        
        
    dict_folder_id_folder_attr[folder_id]["gini_num_days_present_over_members"]=gini(list_days_present_members)

    
    dict_folder_id_folder_attr[folder_id]["eff_num_members_by_days_present"]=effective_num(list_days_present_members)
    
    
    ########### calculate burstiness for the folder's activity:
    
    if len(df_select_folder_act)>2:        
        
        #print 
        #print df_select_folder_act
        list_deltas= ( df_select_folder_act.date - df_select_folder_act.date.shift() ).fillna(0)
        lista_interevent_times=[x.days for x in list_deltas][1:]      # the first delta time is always zero because it comes from comparing to nan when shifting the column


       # print lista_interevent_times
        m1=np.mean(lista_interevent_times)
        s1=np.std(lista_interevent_times)

        burstiness= (s1-m1)/(s1+m1)
        dict_folder_id_folder_attr[folder_id]["burstiness"]= burstiness
        dict_folder_id_folder_attr[folder_id]["avg_interevent_time"]= np.mean(lista_interevent_times)
       
      
 
    else:
        dict_folder_id_folder_attr[folder_id]["burstiness"]=np.nan
        dict_folder_id_folder_attr[folder_id]["avg_interevent_time"]= np.nan
           
        
        
   
    
    
    #### fract of days when multiple users are working simultaneously
    cont_simult=0.
#     if cont >= 10000:
#         break       
    for date in df_select_folder_act.date.unique():
        
        df_select_date = df_select_folder_act[df_select_folder_act['date'] == date]
        num_users=len(df_select_date.user_id.unique())
        if num_users >1:
             cont_simult +=1.
    
    try:
        dict_folder_id_folder_attr[folder_id]["frac_simult_days_overall"] = cont_simult /  dict_folder_id_folder_attr[folder_id]["act_period"]
    except ZeroDivisionError:
        dict_folder_id_folder_attr[folder_id]["frac_simult_days_overall"] = np.nan
        
        
    try:    
        dict_folder_id_folder_attr[folder_id]["frac_simult_days_over_active_days"] = cont_simult /  dict_folder_id_folder_attr[folder_id]["overall_num_active_days"]
    except ZeroDivisionError:
        dict_folder_id_folder_attr[folder_id]["frac_simult_days_over_active_days"] = np.nan
    
    
    
    
#     print "fract simult days:", dict_folder_id_folder_attr[folder_id]["frac_simult_days"], "  cont_sim:",cont_simult, " act_period:",dict_folder_id_folder_attr[folder_id]["act_period"]
   
#     print "overall num. active days in folder:", dict_folder_id_folder_attr[folder_id]["overall_num_active_days"]
#     raw_input()    

    
#         #### i identify who is the most active user using a rolling window              
#         df_rolling_avg=df_act_user.set_index('date').rolling(rolling_window, center=True).mean()       
#         df_rolling_avg['date'] = df_rolling_avg.index
    
    
    
    
    
       


    
    
  
    
    
############################################
#############################################
 

    
    
cont=1

for folder_id in tqdm_notebook( dict_folder_id_list_users):  
  
    list_users=dict_folder_id_list_users[folder_id]

    
    
    df_select_folder_act=df_all_act_no_NANs[df_all_act_no_NANs['folder_id']== folder_id].sort_values(by='date')        
    #df_select_folder_act=df_all_act[df_all_act['folder_id']== folder_id].sort_values(by='date')        

    
    
    #df_select_users_attr= df_user_basic_attr[df_user_basic_attr.user_id.isin(list_users)]        

   

    cont +=1
   # if len(df_select_folder_act)>0:
    grouping_folder(folder_id, dict_folder_id_folder_attr, df_select_folder_act, list_users)


#     if cont >= 10000:
#         break          
  
    
print "done"





#dict_folder_id_folder_attr['folder_id']["gini_num_days_present_over_members"]
    



pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr_with_time_attr.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_folder_id_folder_attr, handle)
print "written:", pickle_name




df_from_dict = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')
## add foler_id from index
df_from_dict.to_csv(pickle_name.replace("dict_",'').strip(".pickle")+".csv", sep=';')
print "written:", pickle_name.replace("dict_","").strip(".pickle")+".csv"


In [ ]:
# import plotly.plotly as py
# import plotly.graph_objs as go

# data = [go.Histogram(x=lista_burstiness_values,
#                      histnorm='probability')]

# py.iplot(data, filename='normalized histogram')
                                                       # REAL DATA:

In [ ]:
lista_b_low=[]
lista_b_high=[]
lista_neutral_b=[]

for folder_id in dict_folder_id_folder_attr:
    if dict_folder_id_folder_attr[folder_id]['overall_num_active_days']>50:
        #print folder
        if dict_folder_id_folder_attr[folder_id]['burstiness']>0.5:   # examples folders: 278921388.0  1022361891.0  640680533.0  500171667.0 94594249.0 1174406155.0 876612829.0
            lista_b_high.append(folder_id)  
        elif dict_folder_id_folder_attr[folder_id]['burstiness'] < -0.5:   # examples folders (rare!): 1288795403.0   1303489998.0   1396197512.0   156510508.0   1175798456.0  184263678.0  23781582.0
            lista_b_low.append(folder_id)
        elif( dict_folder_id_folder_attr[folder_id]['burstiness'] < 0.1)  and   (dict_folder_id_folder_attr[folder_id]['burstiness'] > -0.1) :   # examples folders : 810549883.0  1304953142.0 430966151.0 914359852.0 1220543920.0 
            lista_neutral_b.append(folder_id)
           
#             for key in sorted(dict_folder_id_folder_attr[folder]):
#                 print key, dict_folder_id_folder_attr[folder][key]
#             raw_input()

In [ ]:
print len(lista_b_high)+ len(lista_b_low)+ len(lista_neutral_b)

In [ ]:
folder_id=1027611958.0

for key in sorted(dict_folder_id_folder_attr[folder_id]):
    print key, dict_folder_id_folder_attr[folder_id][key]

In [ ]:
#df_all_act[ df_all_act["folder_id"] == 125829122.0]   #sort_values(by=["user_id","date"]).head(300)
df_all_act[ df_all_act["user_id"] == 125829122.0]   #sort_values(by=["user_id","date"]).head(300)




# 3488229	2015-11-23	2015-03-23	864026646.0	0.0	0.0	3.0	44588732.0   x
# 3488230	2015-11-24	2015-03-23	864026646.0	1.0	1.0	0.0	44588732.0   x
# 3488231	2015-11-25	2015-03-23	864026646.0	0.0	1.0	1.0	44588732.0  x
# 3488232	2015-12-04	2015-03-23	864026646.0	6.0	3.0	0.0	44588732.0 x
# 3488233	2015-12-05	2015-03-23	864026646.0	1.0	0.0	0.0	44588732.0 x
# 3488234	2015-12-09	2015-03-23	864026646.0	1.0	0.0	0.0	44588732.0 x
# 3488235	2015-12-10	2015-03-23	864026646.0	0.0	1.0	0.0	44588732.0 x
# 3488236	2016-02-08	2015-03-23	864026646.0	2.0	1.0	0.0	44588732.0 x
# 3488237	2016-02-09	2015-03-23	864026646.0	0.0	0.0	2.0	44588732.0